In [1]:
# %%
# pip install matplotlib dotenv sentinelhub geopandas IProgress 

# %%
import os
import time
import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
from sentinelhub import SHConfig, SentinelHubCatalog, BBox, CRS, DataCollection
from sentinelhub import MimeType, SentinelHubDownloadClient, SentinelHubRequest, bbox_to_dimensions, filter_times
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import json
import uuid
import glob

# Load environment variables
load_dotenv()

# Configure Sentinel Hub
config = SHConfig()
config.sh_client_id = os.getenv("SENTINELHUB_CLIENT_ID2")
config.sh_client_secret = os.getenv("SENTINELHUB_CLIENT_SECRET2")
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

# Initialize the catalog
catalog = SentinelHubCatalog(config=config)

# Helper function to normalize image based on Scene Classification Layer
def normalize_by_scl(image, scl, valid_scl_values=[4, 5]):
    """
    Normalize image using percentile scaling over valid SCL mask.
    Returns normalized image, valid mask, list of p2, and p98 per band.
    """
    valid_mask = np.isin(scl, valid_scl_values)
    norm_image = np.zeros_like(image, dtype=np.float32)
    norm_p2, norm_p98 = [], []

    for band in range(image.shape[2]):
        band_data = image[:, :, band]
        band_valid = band_data[valid_mask]
        if band_valid.size > 0:
            p2, p98 = np.percentile(band_valid, (2, 98))
        else:
            p2, p98 = 0, 1
        norm_p2.append(float(p2))
        norm_p98.append(float(p98))
        band_data = np.clip(band_data, p2, p98)
        band_data = (band_data - p2) / (p98 - p2 + 1e-6)
        norm_image[:, :, band] = band_data

    return norm_image, valid_mask, norm_p2, norm_p98

# Function to save a single image right after download
def save_image(image, patch_id, timestamp, resolution, base_raw_dir, base_proc_dir, logfile_path, log_df, 
               band_list, original_band_res, patch_bbox, bands_units, bands_num, sampleType, cloud_cover_filter):
    """
    Save a single downloaded Sentinel-2 image and its metadata.
    
    Args:
        image: The raw image data
        patch_id: ID of the patch
        timestamp: Timestamp of the image
        resolution: Resolution of the image
        base_raw_dir: Directory to save raw images
        base_proc_dir: Directory to save processed images
        logfile_path: Path to the log file
        log_df: DataFrame containing log information
        band_list: List of bands in the image
        original_band_res: Dictionary of original resolutions for each band
        patch_bbox: Bounding box of the patch
        bands_units: Units of the bands
        bands_num: Number of bands
        sampleType: Sample type of the image
        cloud_cover_filter: Maximum cloud cover percentage
        
    Returns:
        Updated log DataFrame
    """
    if image is None:
        print(f"Skipping save for {patch_id}/{timestamp} - download failed.")
        return log_df
        
    # Generate unique filename
    short_uuid = str(uuid.uuid4())[:8]
    iso_time = timestamp.isoformat().replace(":", "").replace("-", "")
    base_filename = f"{patch_id}_{iso_time}_res{resolution}"

    # File paths
    raw_tiff_path = os.path.join(base_raw_dir, f"{base_filename}.tiff")
    proc_tiff_path = os.path.join(base_proc_dir, f"{base_filename}.tiff")
    json_path = os.path.join(base_proc_dir, f"{base_filename}.json")  # metadata with processed version

    # Bounding box info for metadata
    ul_lon = round(patch_bbox.lower_left[0], 4)
    ul_lat = round(patch_bbox.upper_right[1], 4)
    br_lon = round(patch_bbox.upper_right[0], 4)
    br_lat = round(patch_bbox.lower_left[1], 4)

    try:
        # Convert to float32 for processing
        image = image.astype(np.float32)

        # Extract SCL and spectral bands
        scl = image[:, :, -1]
        spectral = image[:, :, :-1]

        # Normalize spectral bands using valid SCL pixels
        valid_scl_values = [4, 5]  # Vegetation and Bare Soils
        norm_image, valid_mask, norm_p2, norm_p98 = normalize_by_scl(spectral, scl, valid_scl_values=valid_scl_values)

        # Save raw TIFF (unmodified reflectance + SCL)
        image_raw = np.transpose(image, (2, 0, 1))  # (bands, height, width)
        tifffile.imwrite(raw_tiff_path, image_raw)
        print(f"Saved RAW: {raw_tiff_path}")

        # Save processed TIFF (normalized reflectance only, excluding SCL)
        image_norm = np.transpose(norm_image, (2, 0, 1))  # (bands, height, width)
        tifffile.imwrite(proc_tiff_path, image_norm)
        print(f"Saved PROCESSED: {proc_tiff_path}")

        # Save metadata
        metadata = {
            "source": "SENTINEL2_L2A",
            "patch_id": patch_id,
            "uuid": short_uuid,
            "timestamp": timestamp.isoformat(),
            "resolution": resolution,
            "sampleType": sampleType,
            "bands_units": bands_units,
            "bands_num": bands_num,
            "bands_shape": image.shape,
            "bands": band_list,
            "bbox": {
                "upper_left": [ul_lat, ul_lon],
                "bottom_right": [br_lat, br_lon]
            },
            "original_band_resolutions": original_band_res,
            "cloud_mask_applied": True,
            "max_search_cloud_cover": cloud_cover_filter,
            "normalization": {
                "percentiles": [2, 98],
                "p2": norm_p2,
                "p98": norm_p98
            },
            "file_raw": os.path.basename(raw_tiff_path),
            "file_processed": os.path.basename(proc_tiff_path),
        }

        with open(json_path, "w") as f:
            json.dump(metadata, f, indent=4)
            print(f"Saved metadata: {json_path}")

        # Append log entry for this image
        log_df = pd.concat([log_df, pd.DataFrame([{
            "patch_id": patch_id,
            "uuid": short_uuid,
            "timestamp": timestamp.isoformat(),
            "file_processed": os.path.basename(proc_tiff_path),
            "status": "success"
        }])], ignore_index=True)

        # Save log immediately
        log_df.to_csv(logfile_path, index=False)
        print(f"Logged patch {patch_id} to {logfile_path}")

        return log_df

    except Exception as e:
        print(f"Error saving image: {e}")
        # Log the failure
        log_df = pd.concat([log_df, pd.DataFrame([{
            "patch_id": patch_id,
            "uuid": short_uuid,
            "timestamp": timestamp.isoformat(),
            "file_processed": os.path.basename(proc_tiff_path) if os.path.exists(proc_tiff_path) else None,
            "status": f"error: {str(e)[:100]}"
        }])], ignore_index=True)
        log_df.to_csv(logfile_path, index=False)
        return log_df

#################################
######### PATCHES LOGIC #########
#################################
# Load patches
patches_gdf = gpd.read_file("../data/patches/highlighted_patches.geojson")
print(f"Loaded {len(patches_gdf)} patches")

# Ensure correct CRS
assert patches_gdf.crs.to_epsg() == 3857, "Expected EPSG:3857 in patches.geojson"

# Extract numeric index from patch_id for sorting
patches_gdf["patch_index"] = patches_gdf["patch_id"].str.extract(r'patch_(\d{5})')[0].astype(int)
patches_gdf = patches_gdf.sort_values("patch_index").reset_index(drop=True)

# Resume from specific patch
start_from_patch = "patch_02149_fa728d65"  # ← Change this to resume from another
if start_from_patch in patches_gdf["patch_id"].values:
    start_index = patches_gdf[patches_gdf["patch_id"] == start_from_patch].index[0]
    patches_gdf = patches_gdf.iloc[start_index:].reset_index(drop=True)
    print(f"Resuming from patch: {start_from_patch}")
else:
    print(f"Start patch '{start_from_patch}' not found. Starting from the beginning.")

# Limit number of patches processed
patch_limit = None # Set to None to process all patches, or a specific number
patches_gdf = patches_gdf.head(patch_limit)
print(f"Processing {len(patches_gdf)} patches...")


######################## SPECIFY PARAMETERS #############################
#########################################################################
##########################################################################
# Define the time range for the search
time_interval = ("2010-01-01", "2030-12-31")
cloud_cover_filter = 20  # max % cloud cover allowed

# Specify the bands to download
bands_num = 13
bands_units = "DN"
sampleType = "uint16"
resolution = 1024 #10 # Band Resolution

# Define band list and original resolutions
band_list = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "SCL"]
original_band_res = {
    "B01": 60, "B02": 10, "B03": 10, "B04": 10,
    "B05": 20, "B06": 20, "B07": 20,
    "B08": 10, "B8A": 20, "B09": 60,
    "B11": 20, "B12": 20, "SCL": 20
}

# Set paths
processed_dir = "../data/processed/sentinel2"
logfile_path = "../data/logs/patch_download_log.csv"
os.makedirs(os.path.dirname(logfile_path), exist_ok=True)

# Define the directory to save the images
save_dir = "../data/raw/sentinel2"
os.makedirs(save_dir, exist_ok=True)

# Configuration
pixel_scaling = 1
base_raw_dir = "../data/raw/sentinel2"
base_proc_dir = "../data/processed/sentinel2"
os.makedirs(base_raw_dir, exist_ok=True)
os.makedirs(base_proc_dir, exist_ok=True)
##########################################################################
##########################################################################
##########################################################################

# Function to check if an image exists for a specific patch and timestamp
def check_image_exists(patch_id, timestamp, resolution, raw_dir, proc_dir):
    """Check if an image already exists in either raw or processed directories."""
    iso_time = timestamp.isoformat().replace(":", "").replace("-", "")
    base_filename = f"{patch_id}_{iso_time}_res{resolution}"
    
    raw_path = os.path.join(raw_dir, f"{base_filename}.tiff")
    proc_path = os.path.join(proc_dir, f"{base_filename}.tiff")
    json_path = os.path.join(proc_dir, f"{base_filename}.json")
    
    # Check if both raw and processed files exist
    return os.path.exists(raw_path) and os.path.exists(proc_path) and os.path.exists(json_path)

# List already processed patch_ids from saved files
existing_files = glob.glob(os.path.join(processed_dir, "patch_*.json"))
processed_patch_ids = set()

for path in existing_files:
    fname = os.path.basename(path)
    if fname.startswith("patch_") and fname.endswith(".json"):
        patch_id = fname.split("_")[1]  # e.g., patch_02149_xxx → 02149
        full_id = "_".join(fname.split("_")[1:3])  # e.g., 02149_xxx
        processed_patch_ids.add(f"patch_{full_id}")

# Ensure the directory exists
os.makedirs(os.path.dirname(logfile_path), exist_ok=True)

# Load or initialize patch log
if os.path.exists(logfile_path):
    log_df = pd.read_csv(logfile_path)
    logged_patch_ids = set(log_df["patch_id"])
    print(f"Loaded log with {len(log_df)} entries")
else:
    log_df = pd.DataFrame(columns=["patch_id", "uuid", "timestamp", "file_processed", "status"])
    logged_patch_ids = set()
    print("Initialized new patch log")

# Define the evalscript for downloading Sentinel-2 images
all_bands_evalscript = f"""
//VERSION=3

function setup() {{
    return {{
        input: [{{
            bands: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12", "SCL"],
            units: "{bands_units}"
        }}],
        output: {{
            bands: {bands_num},
            sampleType: "{sampleType}"
        }}
    }};
}}

function evaluatePixel(sample) {{
    return [
        sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06,
        sample.B07, sample.B08, sample.B8A, sample.B09, sample.B11, sample.B12,
        sample.SCL
    ];
}}
"""

##########################
####### MAIN LOOP ########
##########################

# Loop over patches
for idx, row in patches_gdf.iterrows():
    patch_id = row["patch_id"]
    geom = row["geometry"]

    if patch_id in processed_patch_ids:
        print(f"Patch {patch_id} already processed. Skipping.")
        log_df = pd.concat([log_df, pd.DataFrame([{
            "patch_id": patch_id,
            "uuid": None,
            "timestamp": None,
            "file_processed": None,
            "status": "skipped"
        }])], ignore_index=True)
        log_df.to_csv(logfile_path, index=False)
        continue

    if geom.geom_type != "Polygon":
        print(f"Skipping non-polygon geometry in patch {patch_id}")
        log_df = pd.concat([log_df, pd.DataFrame([{
            "patch_id": patch_id,
            "uuid": None,
            "timestamp": None,
            "file_processed": None,
            "status": "non-polygon",
        }])], ignore_index=True)
        log_df.to_csv(logfile_path, index=False)
        continue

    # Convert geometry bounds to a BBox in Web Mercator
    patch_bbox = BBox(bbox=geom.bounds, crs=CRS.POP_WEB)  # EPSG:3857 == POP_WEB

    print(f"\nProcessing patch {patch_id} with patch_bbox: {patch_bbox}")

    # Perform a search within the bounding box and time range
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=patch_bbox,
        time=time_interval,
        filter= f"eo:cloud_cover < {cloud_cover_filter}",  # Optional filter for cloud cover
        fields={"include": ["id", "properties.datetime", "properties.eo:cloud_cover", "geometry"], "exclude": []},
    )

    # Convert the results to a list of features
    features = list(search_iterator)

    # Create a GeoDataFrame from the features
    results_gdf = gpd.GeoDataFrame.from_features(features)

    # Display the results
    print("Total number of results:", len(features))
    if len(features) > 0:
        print(results_gdf)

    # Find unique acquisitions
    time_difference = dt.timedelta(hours=1)
    all_timestamps = search_iterator.get_timestamps()
    unique_acquisitions = filter_times(all_timestamps, time_difference)

    if not unique_acquisitions:
        print(f"No acquisitions found for patch {patch_id}. Skipping.")
        log_df = pd.concat([log_df, pd.DataFrame([{
            "patch_id": patch_id,
            "uuid": None,
            "timestamp": None,
            "file_processed": None,
            "status": "no_acquisitions"
        }])], ignore_index=True)
        log_df.to_csv(logfile_path, index=False)
        continue

    # print('unique_acquisitions: ', unique_acquisitions)

    # Initialize Sentinel Hub client
    client = SentinelHubDownloadClient(config=config)
    
    # Process each unique acquisition
    for timestamp in unique_acquisitions:
        # Check if this image already exists in both raw and processed directories
        if check_image_exists(patch_id, timestamp, resolution, base_raw_dir, base_proc_dir):
            print(f"Image for {patch_id} at {timestamp} already exists. Skipping.")
            continue
            
        # Create request for this timestamp
        request = SentinelHubRequest(
            evalscript=all_bands_evalscript,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A.define_from("s2l2a", service_url=config.sh_base_url),
                    time_interval=(timestamp - time_difference, timestamp + time_difference),
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
            bbox=patch_bbox,
            size=(resolution, resolution),  # Force fixed pixel dimensions
            config=config,
        )
        
        # Download with retry logic
        max_retries = 5
        image_data = None
        
        for attempt in range(max_retries):
            try:
                print(f"Downloading image for {patch_id} at {timestamp.isoformat()} (attempt {attempt+1}/{max_retries})...")
                download_request = request.download_list[0]
                image_data = client.download(download_request)
                break
            except Exception as e:
                is_throttle = any(code in str(e) for code in ["429", "503"])
                wait = 2 ** attempt
                print(f"⚠️ Attempt {attempt+1} failed: {e}")
                if is_throttle:
                    print(f"⏳ Throttled! Waiting {wait}s and retrying...")
                    time.sleep(wait)
                elif attempt < max_retries - 1:
                    print(f"Waiting {wait}s and retrying...")
                    time.sleep(wait)
                else:
                    print(f"⛔️ All attempts failed for {timestamp}. Skipping.")
                    # Log failure
                    log_df = pd.concat([log_df, pd.DataFrame([{
                        "patch_id": patch_id,
                        "uuid": str(uuid.uuid4())[:8],
                        "timestamp": timestamp.isoformat(),
                        "file_processed": None,
                        "status": f"download_failed: {str(e)[:100]}"
                    }])], ignore_index=True)
                    log_df.to_csv(logfile_path, index=False)
        
        # Save the image if download was successful
        if image_data is not None:
            print(f"Successfully downloaded image for {patch_id} at {timestamp.isoformat()}")
            if len(image_data.shape) == 3:  # Ensure we have a valid 3D array
                log_df = save_image(
                    image_data, patch_id, timestamp, resolution, 
                    base_raw_dir, base_proc_dir, logfile_path, log_df,
                    band_list, original_band_res, patch_bbox, bands_units, 
                    bands_num, sampleType, cloud_cover_filter
                )
            else:
                print(f"⚠️ Invalid image data shape: {image_data.shape}")
                
        # Throttle between requests to avoid hitting rate limits
        time.sleep(1.0)

    print(f"Finished processing patch {patch_id}")

print("Processing complete!")
# %%

/home/fox/geo-ai-superres/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 16 patches
Start patch 'patch_02149_fa728d65' not found. Starting from the beginning.
Processing 16 patches...
Loaded log with 1433 entries
Patch patch_00032_5925311e already processed. Skipping.
Patch patch_00033_4e36ed13 already processed. Skipping.
Patch patch_00034_1558c231 already processed. Skipping.
Patch patch_00035_2dffd3d8 already processed. Skipping.
Patch patch_00041_6f274c8a already processed. Skipping.
Patch patch_00042_e6bff2f8 already processed. Skipping.
Patch patch_00043_691c8352 already processed. Skipping.
Patch patch_00044_4359be37 already processed. Skipping.

Processing patch patch_00050_b75fc6d6 with patch_bbox: 12828046.745621826,-969261.6018141088,12838449.18481961,-958790.6375444453


/tmp/ipykernel_16082/1282372795.py:355: SHDeprecationWarning: The string representation of `BBox` will change to match its `repr` representation.
  print(f"\nProcessing patch {patch_id} with patch_bbox: {patch_bbox}")


Total number of results: 345
                                              geometry  \
0    MULTIPOLYGON (((114.91545 -8.13669, 115.27351 ...   
1    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
2    MULTIPOLYGON (((114.90647 -8.13665, 115.27351 ...   
3    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
4    MULTIPOLYGON (((115.2712 -8.66007, 115.26899 -...   
..                                                 ...   
340  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
341  MULTIPOLYGON (((114.92189 -8.13673, 115.27351 ...   
342  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
343  MULTIPOLYGON (((114.91464 -8.13669, 115.27351 ...   
344  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   

                     datetime  eo:cloud_cover  
0    2025-05-11T02:40:34.345Z           18.62  
1    2025-05-11T02:40:31.545Z           14.95  
2    2025-05-01T02:40:37.624Z           16.59  
3    2025-05-01T02:40:34.822Z           18.60  
4    2025-04-26T02:40:20.181Z     

/home/fox/geo-ai-superres/venv/lib/python3.12/site-packages/sentinelhub/download/client.py:79: SHDeprecationWarning: The parameter `download_requests` should be a sequence of requests. In future versions download of single requests will only be supported if provided as a singelton tuple or list.
  warnings.warn(


...

Successfully downloaded image for patch_00059_6ea9e5b3 at 2015-10-21T02:40:17.632000+00:00
Saved RAW: ../data/raw/sentinel2/patch_00059_6ea9e5b3_20151021T024017.632000+0000_res1024.tiff
Saved PROCESSED: ../data/processed/sentinel2/patch_00059_6ea9e5b3_20151021T024017.632000+0000_res1024.tiff
Saved metadata: ../data/processed/sentinel2/patch_00059_6ea9e5b3_20151021T024017.632000+0000_res1024.json
Logged patch patch_00059_6ea9e5b3 to ../data/logs/patch_download_log.csv
Successfully downloaded image for patch_00059_6ea9e5b3 at 2016-03-09T02:40:07.993000+00:00
Saved RAW: ../data/raw/sentinel2/patch_00059_6ea9e5b3_20160309T024007.993000+0000_res1024.tiff
Saved PROCESSED: ../data/processed/sentinel2/patch_00059_6ea9e5b3_20160309T024007.993000+0000_res1024.tiff
Saved metadata: ../data/processed/sentinel2/patch_00059_6ea9e5b3_20160309T024007.993000+0000_res1024.json
Logged patch patch_00059_6ea9e5b3 to ../data/logs/patch_download_log.csv
Successfully downloaded image for patch_00059_6ea9e5b3 a

/tmp/ipykernel_16082/1282372795.py:355: SHDeprecationWarning: The string representation of `BBox` will change to match its `repr` representation.
  print(f"\nProcessing patch {patch_id} with patch_bbox: {patch_bbox}")


Total number of results: 192
                                              geometry  \
0    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
1    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
2    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
3    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
4    MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
..                                                 ...   
187  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
188  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
189  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
190  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   
191  MULTIPOLYGON (((115.18479 -8.13799, 116.18122 ...   

                     datetime  eo:cloud_cover  
0    2025-05-11T02:40:31.545Z           14.95  
1    2025-05-01T02:40:34.822Z           18.60  
2    2025-04-23T02:40:27.062Z           15.62  
3    2025-04-03T02:40:24.469Z           11.76  
4    2025-03-02T02:40:37.503Z     

/home/fox/geo-ai-superres/venv/lib/python3.12/site-packages/sentinelhub/download/client.py:79: SHDeprecationWarning: The parameter `download_requests` should be a sequence of requests. In future versions download of single requests will only be supported if provided as a singelton tuple or list.
  warnings.warn(


Successfully downloaded image for patch_00060_4ced72c0 at 2015-10-21T02:40:17.632000+00:00
Saved RAW: ../data/raw/sentinel2/patch_00060_4ced72c0_20151021T024017.632000+0000_res1024.tiff
Saved PROCESSED: ../data/processed/sentinel2/patch_00060_4ced72c0_20151021T024017.632000+0000_res1024.tiff
Saved metadata: ../data/processed/sentinel2/patch_00060_4ced72c0_20151021T024017.632000+0000_res1024.json
Logged patch patch_00060_4ced72c0 to ../data/logs/patch_download_log.csv
Successfully downloaded image for patch_00060_4ced72c0 at 2016-03-09T02:40:07.993000+00:00
Saved RAW: ../data/raw/sentinel2/patch_00060_4ced72c0_20160309T024007.993000+0000_res1024.tiff
Saved PROCESSED: ../data/processed/sentinel2/patch_00060_4ced72c0_20160309T024007.993000+0000_res1024.tiff
Saved metadata: ../data/processed/sentinel2/patch_00060_4ced72c0_20160309T024007.993000+0000_res1024.json
Logged patch patch_00060_4ced72c0 to ../data/logs/patch_download_log.csv
Successfully downloaded image for patch_00060_4ced72c0 a